# **Previsão de preço de carros.**
<!--
<img src="https://miro.medium.com/v2/resize:fit:988/0*tA5OjppLK627FfFo" alt="image" width="250" height="auto">
-->
Nesse projeto eu vou construir um modelo de **Machine Learning** que faça a previsão  do preço de venda de um veículo.

Irei utilizar a Metodologia **CRISP-DM** para me basear nos passos a serem seguinos no projeto.

1. **Compreensão do Negócio (Business Understanding/Problem)** 

1. **Compreensão dos Dados (Data Understanding)** 
1. **Preparação dos Dados (Data Preparation)** 
1. **EDA**
1. **Modelagem (Modeling)** 
1. **Avaliação (Evaluation)**
1. **Implantação (Deployment)** 

# **1. Problema de Negóscio**

*   Uma pessoa precisa revender seu veículo porém ela **não sabe** qual valor deve cobrar no mesmo.
*   1.1 Qual é o contexto?
    *   ...
*   1.2 Quais são os objetivos do projeto?
    *   Identificar as características que influenciam os preços de venda dos veículos.
    *   Criar um modelo capaz de prever o valor de venda de um novo veículo.
    *   ...
*   1.3 Quais são os benefícios?
    *   Vender o veículo com o valor *ótimo* de acordo com **suas características**.
    *   **Reduzir as perdas de dinheiro** na revenda do veículo.
    *   **Enconomizar esforços de energia/tempo** em tentar adivinhar o valor ideal.

Como resultado, o **problema de negócio** será resolvido.

### **Importando as bibliotecas necessárias.**

In [1]:
from platform import python_version
print('Versão da Linguagem Python:', python_version())

Versão da Linguagem Python: 3.12.5


In [2]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
# import matplotlib as mpl
import seaborn as sns

# Display options.
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_info_rows', 100)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format
# pd.set_option('display.width', 1000)

# **2. Compreendendo os dados**
*   Os dados foram obtidos através de web scraping do site  [Olx](https://www.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios?) e **são de propiedade de olx.com**
*   O repositório do projeto de web scraping completo se encontra [aqui](https://github.com/PatrickLeal/olx-veiculos-web-scraping).
*   Os dados contém informações sobre os veículos anunciados no site. 

In [3]:
file_path = glob.glob('../data_lake/silver/*.csv')
df_carros = pd.read_csv(file_path[0], parse_dates=['DATA_PUBLICACAO'])           

In [4]:
df_carros.sample(3)

,TITULO,PRECO_BRL,DESCRICAO,OPCIONAIS,CEP,IMAGEM,PERFIL_CARRO,DATA_PUBLICACAO,TEMPO_PUBLICACAO,COD_PUBLICACAO,TIPO_ANUNCIO,MODELO,PORTAS,CATEGORIA,CAMBIO,COR,POTENCIA_DO_MOTOR,QUILOMETRAGEM,TIPO_DE_DIREÇÃO,COMBUSTIVEL,FINAL_DE_PLACA,MARCA,ANO,TIPO_DE_VEICULO,POSSUI_KIT_GNV
6428,JEEP GRAND CHEROKEE 2.0 TURBO HÍBRIDO 4XE AT8,499990.00,CONDIÇÃO PARA PESSOA FÍSICA OFERTA EXCLUSI...,[],54330000,https://img.olx.com.br/images/83/8364255607711...,https://pe.olx.com.br/grande-recife/autos-e-pe...,2024-09-10,11:08,1336422692,profissional,JEEP GRAND CHEROKEE,4 portas,"Carros, vans e utilitários",Automático,Cinza,2.0 - 2.9,0,NaN,NaN,NaN,JEEP,2023,NaN,Não
6587,Up 2015 Completo,36900.00,"Up ano 2015 , veículo terceiro dono , todo rev...","['Alarme', 'Air bag', 'Ar condicionado', 'Som'...",36400050,https://img.olx.com.br/images/97/9724242066933...,https://mg.olx.com.br/belo-horizonte-e-regiao/...,2024-09-10,11:07,1336422099,profissional,VOLKSWAGEN UP! MOVE 1.0 TOTAL FLEX 12V 5P,4 portas,"Carros, vans e utilitários",Manual,Branco,1.0,150000,Hidráulica,Flex,2.00,VOLKSWAGEN,2015,Hatch,Não
15314,Audi A3 Sportback 2.0 16V TFSI S-Tronic,62900.00,marcos3198... ver número Quilometragem 171.349...,"['Alarme', 'Air bag', 'Ar condicionado', 'Banc...",31140540,https://img.olx.com.br/images/45/4584756842663...,https://mg.olx.com.br/belo-horizonte-e-regiao/...,2024-09-17,11:09,1335362684,profissional,AUDI A3 SPORTBACK 2.0 16V TFSI S-TRONIC,4 portas,"Carros, vans e utilitários",Automático,Preto,2.0 - 2.9,171349,Elétrica,Gasolina,NaN,AUDI,2011,Hatch,Não


### **Dicionário de dados**

|VARIÁVEL| DESCRIÇÃO | TIPO |
|--------|-----------|------|
|**TITULO:**| Título do anúncio| string |
|**PRECO_BRL:**| Preço em Reais R$| int |
|**DESCRICAO:**| Descrição do anúncio deixada pelo anunciante| string |
|**OPCIONAIS:**| São *features* dos veículos informadas pelo anunciante| string |
|**CEP:**| Localização veículo deixada pelo anunciante | int |
|**IMAGEM:**| Link da foto de perfil do anúncio | string |
|**PERFIL_CARRO:**| Link para o anúncio original | string |
|**DATA_PUBLICACAO:**| Date em que o anúncio foi criado| date |
|**TEMPO_PUBLICACAO:**| Hora em que o anúncio foi criado| time |
|**COD_PUBLICACAO:**| Código do anúncio| int |
|**TIPO_ANUNCIO:**| Tipo de anúncio| string|
|**ANO:**| Informa o Ano doveículo| int |
|**MARCA:**| Nome da marca do veículo| string |
|**CAMBIO:**| Tipo de câmbio do veículo | string |
|**POSSUI_KIT_GNV:**| Se o veículo possui kit GNV| bool |
|**PORTAS:**| Quantidade de portas que o veículo possui| int |
|**POTENCIA_DO_MOTOR:**| Informa a potência do motor | string |
|**TIPO_DE_VEICULO:**| Informa o tipo de veículo |string |
|**FINAL_DE_PLACA:**| Informa o final da placa| int |
|**COR:**| Informa a cor do veículo| string |
|**QUILOMETRAGEM:**| Informa a quilometragem do veículo| int |
|**COMBUSTIVEL:**| Informa o o tipo de combustível usado pelo veículo| string|
|**TIPO_DE_DIREÇÃO:**| Informa o tipo de direção do veículo| string |
|**MODELO:**| Informa o modelo do veículo|string |
|**CATEGORIA:**| Informa e qual categoria o veículo foi anunciado |string |

`PRECO_BRL`  será nossa variável ***target***

### **Informações gerais dos dados:**

In [5]:
df_carros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18805 entries, 0 to 18804
Data columns (total 25 columns):
 #   Column             Dtype         
---  ------             -----         
 0   TITULO             object        
 1   PRECO_BRL          float64       
 2   DESCRICAO          object        
 3   OPCIONAIS          object        
 4   CEP                int64         
 5   IMAGEM             object        
 6   PERFIL_CARRO       object        
 7   DATA_PUBLICACAO    datetime64[ns]
 8   TEMPO_PUBLICACAO   object        
 9   COD_PUBLICACAO     int64         
 10  TIPO_ANUNCIO       object        
 11  MODELO             object        
 12  PORTAS             object        
 13  CATEGORIA          object        
 14  CAMBIO             object        
 15  COR                object        
 16  POTENCIA_DO_MOTOR  object        
 17  QUILOMETRAGEM      int64         
 18  TIPO_DE_DIREÇÃO    object        
 19  COMBUSTIVEL        object        
 20  FINAL_DE_PLACA     float64  

In [6]:
print(f"O dataset possui {df_carros.shape[0]} linhas e {df_carros.shape[1]} colunas.")

O dataset possui 18805 linhas e 25 colunas.


**Visualisando algumas estatísticas:**

In [7]:
df_carros.describe().T

,count,mean,min,25%,50%,75%,max,std
PRECO_BRL,18644.00,88322.99,0.00,52990.00,71990.00,105900.00,4950000.00,75600.07
CEP,18805.00,39824387.67,1013010.00,12040001.00,31160190.00,68040050.00,99680000.00,29547174.27
DATA_PUBLICACAO,18805,2024-09-12 12:47:30.901356032,2024-07-12 00:00:00,2024-09-09 00:00:00,2024-09-13 00:00:00,2024-09-17 00:00:00,2024-09-17 00:00:00,NaN
COD_PUBLICACAO,18805.00,1329885212.11,614262154.00,1329351445.00,1334684764.00,1336423149.00,1338383745.00,18651014.29
QUILOMETRAGEM,18805.00,72242.72,0.00,43502.00,53219.00,92094.00,9999999.00,107297.16
FINAL_DE_PLACA,15763.00,4.40,0.00,2.00,4.00,7.00,9.00,2.87


**Análise:**
-   o dataset possui anúncios com o **preço** de venda **igual a 0**, o que não faz sentido nesse contexto
-   alguns dados estão com **tipos errados** e precisam ser tradados

**Verificando valores nulos e duplicados:**

In [8]:
print("Porcentagem de valores nulos.")
(df_carros.isnull().sum() / df_carros.shape[0]) * 100

Porcentagem de valores nulos.


TITULO               0.00
PRECO_BRL            0.86
DESCRICAO            0.00
OPCIONAIS            0.00
CEP                  0.00
IMAGEM               0.00
PERFIL_CARRO         0.00
DATA_PUBLICACAO      0.00
TEMPO_PUBLICACAO     0.00
COD_PUBLICACAO       0.00
TIPO_ANUNCIO        24.53
MODELO               0.00
PORTAS               1.75
CATEGORIA            0.00
CAMBIO               0.44
COR                  1.80
POTENCIA_DO_MOTOR    4.53
QUILOMETRAGEM        0.00
TIPO_DE_DIREÇÃO      6.47
COMBUSTIVEL          0.46
FINAL_DE_PLACA      16.18
MARCA                0.00
ANO                  0.00
TIPO_DE_VEICULO     11.79
POSSUI_KIT_GNV       0.00
dtype: float64

In [9]:
print(f"Quantidade de valores duplicados: {df_carros.duplicated().sum()}")

Quantidade de valores duplicados: 2789


**Ações a serem tomada:**
*   remover duplicatas
*   remover colunas que tenham mais de 15% de valores nulos
*   remover coluna *CATEGORIA* porque ela só possui 1  valor único
*   alterar os tipos de dados das colunas:
    1. ANO
    1. PORTAS
    1. POTENCIA_DO_MOTOR
    1. OPCIONAIS
*   remover valores nulos

In [10]:
import os
import sys

module_path = os.path.abspath(os.getcwd() + '//..')
if module_path not in sys.path:
    sys.path.append(module_path)

from scripts import cleaning_tool

df_cleaned = cleaning_tool.limpar_dados_silver(df_carros)

In [11]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12477 entries, 0 to 12476
Data columns (total 23 columns):
 #   Column             Dtype         
---  ------             -----         
 0   titulo             object        
 1   preco_brl          int64         
 2   descricao          object        
 3   opcionais          object        
 4   cep                category      
 5   imagem             object        
 6   perfil_carro       object        
 7   data_publicacao    datetime64[ns]
 8   tempo_publicacao   object        
 9   cod_publicacao     int64         
 10  modelo             object        
 11  portas             int64         
 12  cambio             object        
 13  cor                object        
 14  potencia_do_motor  category      
 15  quilometragem      int64         
 16  tipo_de_direção    object        
 17  combustivel        object        
 18  marca              object        
 19  ano                int64         
 20  tipo_de_veiculo    object   

In [12]:
# df_carros.drop_duplicates(inplace=True)
print("Porcentagem de valores nulos.")
(df_cleaned.isnull().sum() / df_carros.shape[0]) * 100

Porcentagem de valores nulos.


titulo              0.00
preco_brl           0.00
descricao           0.00
opcionais           0.00
cep                 0.00
imagem              0.00
perfil_carro        0.00
data_publicacao     0.00
tempo_publicacao    0.00
cod_publicacao      0.00
modelo              0.00
portas              0.00
cambio              0.00
cor                 0.00
potencia_do_motor   0.00
quilometragem       0.00
tipo_de_direção     0.00
combustivel         0.00
marca               0.00
ano                 0.00
tipo_de_veiculo     0.00
possui_kit_gnv      0.00
qtd_opcionais       0.00
dtype: float64

Agora não há mais dados nulos e seus tipos estão corretos.

# **3. Dividir os dados em treino e teste**
*   A **análise exploratória de dados** será feita nos dados de treino na intenção de evitar *data leakage*
*   O objetivo principal da divisão é verificar o quão bem o modelo pode fazer previsões em **dados que ele nunca viu antes**. 
*   Se o modelo for avaliado apenas nos **dados de treino**, o resultado pode ser tendencioso, pois o modelo já viu aqueles dados.

Removendo variáveis que não importam para nossa análise tendo como **premissa** que tais variáveis **não influenciam** na precificação do veículo.

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
df_cleaned.head(3)

,titulo,preco_brl,descricao,opcionais,cep,imagem,perfil_carro,data_publicacao,tempo_publicacao,cod_publicacao,modelo,portas,cambio,cor,potencia_do_motor,quilometragem,tipo_de_direção,combustivel,marca,ano,tipo_de_veiculo,possui_kit_gnv,qtd_opcionais
0,Fiat Mobi 2023 1.0 evo flex like. manual,53590,FIAT MOBI 1.0 EVO FLEX LIKE. MANUAL - 2023 / 2...,"Ar condicionado, Trava elétrica",71200020,https://img.olx.com.br/images/46/4634813239611...,https://df.olx.com.br/distrito-federal-e-regia...,2024-09-09,19:21,1335272425,FIAT MOBI LIKE 1.0 FIRE FLEX 5P.,4,Manual,Branco,1.0,45021,Hidráulica,Flex,FIAT,2023,Hatch,Não,2
1,Omega CD 3.0,39900,Carro conservado tem potencial Completo Teto s...,"Alarme, Air bag, Ar condicionado, Bancos de co...",8030070,https://img.olx.com.br/images/98/9874565435056...,https://sp.olx.com.br/sao-paulo-e-regiao/autos...,2024-09-09,19:21,1260360322,CHEVROLET OMEGA CD 4.1 / 3.0,4,Manual,Azul,4.0 ou mais,156000,Hidráulica,Gasolina,CHEVROLET,1994,Sedã,Não,8
2,VW Fox Peper Troco Por maior valor,52000,VW Fox Peper 1.6 o mais top da categoria impec...,"Alarme, Air bag, Ar condicionado, Câmera de ré...",13203280,https://img.olx.com.br/images/19/1944761991382...,https://sp.olx.com.br/sao-paulo-e-regiao/autos...,2024-09-09,19:21,1333110058,VOLKSWAGEN FOX PEPPER I MOTION 1.6 FLEX 16V 5P,4,Automatizado,Prata,1.6,88700,Elétrica,Flex,VOLKSWAGEN,2017,Hatch,Não,15


In [15]:
df_filtrado = df_cleaned.drop(columns=['imagem', 'perfil_carro', 'tempo_publicacao', 'cod_publicacao',
                                       'titulo', 'descricao']).copy()
df_filtrado = df_filtrado.drop_duplicates()
X = df_filtrado.drop(columns=['preco_brl'])
y = df_filtrado['preco_brl'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [16]:
print(f"Shape do dataset predictor de treino: {X_train.shape}")
print(f"Shape do dataset target de treino: {y_train.shape}")
print(f"Shape do dataset predictor de teste: {X_test.shape}")
print(f"Shape do dataset target de teste: {y_test.shape}")

Shape do dataset predictor de treino: (8710, 16)
Shape do dataset target de treino: (8710,)
Shape do dataset predictor de teste: (3733, 16)
Shape do dataset target de teste: (3733,)


# **4. Explorando os dados (EDA)**

Eu vou explorar os dados em busca de entender melhor como as variáveis se comportam e também tentar **obter algum insight** com os dados.

Antes eu preciso juntar novamente as variáveis descritivas junto com a variável *target*, porém apenas do dataset de **treino**.

In [17]:
train = pd.concat([X_train, y_train], axis=1)
train.sample(3, random_state=42)

,opcionais,cep,data_publicacao,modelo,portas,cambio,cor,potencia_do_motor,quilometragem,tipo_de_direção,combustivel,marca,ano,tipo_de_veiculo,possui_kit_gnv,qtd_opcionais,preco_brl
7296,"Alarme, Air bag, Ar condicionado, Câmera de ré...",22271110,2024-09-13,VOLKSWAGEN GERAÇÃO VII 1.6 8V FLEX MEC. 4P,4,Manual,Prata,1.6,40000,Hidráulica,Flex,VOLKSWAGEN,2019,Hatch,Sim,14,45999
8202,"Ar condicionado, Trava elétrica",8115460,2024-09-13,FIAT MOBI LIKE 1.0 FIRE FLEX 5P.,4,Manual,Preto,1.0,53893,Hidráulica,Flex,FIAT,2023,Hatch,Não,2,51790
9471,"Som, Bancos de couro, Vidro elétrico",42700000,2024-09-17,RENAULT CLIO RL / YAHOO/ AUTHENT. 1.0 8V 5P,4,Manual,Preto,1.0,150,Mecânica,Gasolina,RENAULT,2002,Hatch,Não,3,4500


Criando listas com as variáveis **categóricas, numéricas e target** para auxiliar na análise.

In [18]:
num_features = X_train.select_dtypes('number').columns.tolist()
cat_features = X_train.select_dtypes(['object', 'category']).columns.tolist()
target = 'preco_brl'

print(f"Há {len(num_features)} variáveis numéricas.\nElas são: {num_features}\n")
print(f"Há {len(cat_features)} variáveis categóricas.\nElas são: {cat_features}")
print(f"\nA variável target será: '{target}'")

Há 4 variáveis numéricas.
Elas são: ['portas', 'quilometragem', 'ano', 'qtd_opcionais']

Há 11 variáveis categóricas.
Elas são: ['opcionais', 'cep', 'modelo', 'cambio', 'cor', 'potencia_do_motor', 'tipo_de_direção', 'combustivel', 'marca', 'tipo_de_veiculo', 'possui_kit_gnv']

A variável target será: 'preco_brl'


In [20]:
for feature in cat_features:
    if feature == 'opcionais':
        unique_opcional = list(train.opcionais[1].split(','))
        for linha in train.opcionais:
            for feat in linha.split(','):
                if feat not in unique_opcional:
                    unique_opcional.append(feat)
        unique_opcional = [op.strip() for op in unique_opcional]            
        unique_opcional = list(dict.fromkeys(unique_opcional))
        print(feature)
        print('-'*40)
        print(f'Há {len(unique_opcional)} valores únicos.')
        print('A coluna "opcionais" precisa ser analisada diferentemente.\n')
        continue
    print(feature)
    print('-'*40)
    print(f'Há {train[feature].nunique()} valores únicos. Eles são: ')
    # print(train[feature].value_counts(normalize=True))
    print()


opcionais
----------------------------------------
Há 17 valores únicos.
A coluna "opcionais" precisa ser analisada diferentemente.

cep
----------------------------------------
Há 3519 valores únicos. Eles são: 

modelo
----------------------------------------
Há 1642 valores únicos. Eles são: 

cambio
----------------------------------------
Há 4 valores únicos. Eles são: 

cor
----------------------------------------
Há 10 valores únicos. Eles são: 

potencia_do_motor
----------------------------------------
Há 12 valores únicos. Eles são: 

tipo_de_direção
----------------------------------------
Há 5 valores únicos. Eles são: 

combustivel
----------------------------------------
Há 6 valores únicos. Eles são: 

marca
----------------------------------------
Há 43 valores únicos. Eles são: 

tipo_de_veiculo
----------------------------------------
Há 10 valores únicos. Eles são: 

possui_kit_gnv
----------------------------------------
Há 2 valores únicos. Eles são: 

